In [1]:
import pandas as pd
import json, os

In [2]:
with open('c:/Temp/PANDASPLOTLY_FUNCODING_FULLDATA_20240601/00_Material(Uploaded)/COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding = 'utf-8-sig')as json_file:
    json_data = json.load(json_file)
def country_name_convert(row):
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']

def creat_dataframe(filename):
    doc = pd.read_csv('c:/Temp/PANDASPLOTLY_FUNCODING_FULLDATA_20240601/00_Material(Uploaded)/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/' + filename, encoding = 'utf-8-sig')
    try:
        doc = doc[['Country_Region','Confirmed']]
    except:
        doc = doc[['Country/Region','Confirmed']]
        doc.columns[['Country_Region','Confirmed']]
    doc.dropna(subset = ['Confirmed'])
    doc['Country_Region'] = doc.apply(country_name_convert, axis = 1)
    doc = doc.astype({'Confirmed' : "int64"})
    doc = doc.groupby('Country_Region').sum()

    data_column = filename.split(".")[0].lstrip('0').replace('-','/')
    doc.columns = [data_column]
    return doc

In [3]:
import os
from datetime import datetime

path = 'c:/Temp/PANDASPLOTLY_FUNCODING_FULLDATA_20240601/00_Material(Uploaded)/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'

In [4]:
def generate_dataframe_by_path(path):
    file_list = os.listdir(path)
    csv_list = list()
    first_doc = True

    for file in file_list:
        if file.split('.')[-1] == 'csv':
            csv_list.append(file)

    csv_list.sort(key = lambda x: datetime.strptime(x, '%m-%d-%Y.csv'))

    for file in csv_list:
        doc = creat_dataframe(file)
        if first_doc:
            final_doc, first_doc = doc, False
        else:
            final_doc = pd.merge(final_doc, doc, how = 'outer', leftindex = True, rightindex = True)

    final_doc = final_doc.fillna(0)
    return final_doc

In [5]:
def crear_flag_link(row):
    flag_link = 'https://public.flourish.studio/country-flags/'+row.lower()+'.svg'
    return flag_link

df_confirmed = generate_dataframe_by_path(path)
df_confirmed = df_confirmed.astype('int64')

country_info = pd.read_csv('c:/Temp/PANDASPLOTLY_FUNCODING_FULLDATA_20240601/00_Material(Uploaded)/COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv', encoding = 'utf-8-sig', keep_default_na=False, na_values= '')
country_info = country_info[['iso2','Country_Region']]
country_info = country_info.drop_duplicates(subset = 'Country_Region', keep = 'last')

doc_final_data = pd.merge(df_confirmed,country_info, how="left", on = 'Country_Region')
doc_final_data = doc_final_data.dropna(subset = 'iso2')
doc_final_data['iso2'] = doc_final_data['iso2'].apply(crear_flag_link)

cols = doc_final_data.columns.tolist()
cols.remove('iso2')
cols.insert(1,'iso2')
doc_final_data = doc_final_data[cols]
cols[1] = 'Country_Flag'
doc_final_data.columns = cols

doc_final_data.to_csv('c:/Temp/PANDASPLOTLY_FUNCODING_FULLDATA_20240601/00_Material(Uploaded)/COVID-19-master/final_covid_data_for_graph.csv')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices